In [53]:
from tbparse import SummaryReader
from pathlib import Path
from ruamel.yaml import YAML
from tqdm.auto import tqdm
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio


yaml = YAML(typ="safe", pure=True)
results_dir = Path("../results/mono-rv")
cache_dir = Path(".cache/naive_test")
pio.kaleido.scope.mathjax = None

In [7]:
test_dirs = [*results_dir.iterdir()]
for test_dir in tqdm(test_dirs):
    dst_path = cache_dir / f"{test_dir.name}.h5"
    if dst_path.exists():
        continue

    board = SummaryReader(test_dir / "board")
    dst_path.parent.mkdir(parents=True, exist_ok=True)
    board.scalars.to_hdf(dst_path, key="scalars")


def read_scalars(test_dir: Path):
    dst_path = cache_dir / f"{test_dir.name}.h5"
    return pd.read_hdf(dst_path, key="scalars")

  0%|          | 0/54 [00:00<?, ?it/s]

In [15]:
dfs = []

for test_dir in results_dir.iterdir():
    with open(test_dir / "config.yml", "r") as f:
        cfg = yaml.load(f)

    assert cfg["env"]["type"] == "atari"
    env_id = cfg["env"]["atari"]["env_id"]
    seed = cfg["repro"]["seed"]

    df = read_scalars(test_dir)
    df = df[df["tag"] == "val/mean_ep_ret"].copy()
    df["env"] = env_id
    df["seed"] = seed
    env_id_, seed_eq, freq_eq = test_dir.name.split("=")
    df["freq"] = int(freq_eq.removeprefix("freq="))
    dfs.append(df)

df = pd.concat(dfs, axis=0)

In [21]:
df[df["env"] == "Pong"]

,step,tag,value,env,seed,freq
297155,24004,val/mean_ep_ret,-20.93750,Pong,0,8
297156,64004,val/mean_ep_ret,-20.21875,Pong,0,8
297157,104004,val/mean_ep_ret,-15.59375,Pong,0,8
297158,144004,val/mean_ep_ret,-14.18750,Pong,0,8
297159,184004,val/mean_ep_ret,-12.56250,Pong,0,8
...,...,...,...,...,...,...
297178,624004,val/mean_ep_ret,-11.46875,Pong,1,8
297179,664004,val/mean_ep_ret,-11.68750,Pong,1,8
297180,704004,val/mean_ep_ret,-10.87500,Pong,1,8
297181,744004,val/mean_ep_ret,-10.06250,Pong,1,8


In [57]:
env_ids = ["Pong", "CrazyClimber", "Assault"]
freqs = [64, 32, 16, 8, 4, 2]

fig = make_subplots(
    rows=len(freqs),
    cols=len(env_ids),
    column_titles=env_ids,
    row_titles=freqs,
    horizontal_spacing=0.1,
    vertical_spacing=0.05,
)

for ix, env_id in enumerate(env_ids, 1):
    df1 = df[df["env"] == env_id]
    for iy, freq in enumerate(freqs, 1):
        df2 = df1[df1["freq"] == freq]
        xpr = px.line(df2, x="step", y="value", color="seed")
        for trace in xpr.data:
            fig.add_trace(trace, iy, ix)

fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1024,
    height=480 * 3,
)

fig.write_image("../tex/assets/naive_perf.pdf")

fig